In [ ]:
import requests
import pandas as pd
from urllib.parse import quote
from dotenv import load_dotenv
import os


load_dotenv()  # 현재 프로젝트 폴더(또는 지정한 경로)에 있는 .env 파일 로드
# 네이버 개발자 센터에서 발급받은 값으로 변경하세요
client_id = os.getenv('NAVER_CLIENT_ID')
client_secret = os.getenv('NAVER_CLIENT_SECRET')

# 검색어
search_query = "LLM"

# 수집할 페이지 수 (예: 10페이지 -> 총 100개)
num_pages = 50

# 결과를 담을 리스트
results = []

for page in range(1, num_pages+1):
    # 네이버 검색 API 문서에 따르면, start 파라미터는 1부터 시작하여
    # 10씩 증가(예: 1, 11, 21, 31, ...)
    start = (page - 1) * 10 + 1
    print(page, end=",")

    # 요청할 URL
    url = "https://openapi.naver.com/v1/search/blog"

    # 요청 헤더
    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }

    # 요청 파라미터
    params = {
        "query": quote(search_query),  # 검색어
        "display": 100,                 # 페이지당 문서 개수
        "start": start,               # 검색 시작 위치
        "sort": "sim"                 # 정렬 옵션("sim": 정확도순, "date": 날짜순)
    }

    # GET 요청
    response = requests.get(url, headers=headers, params=params)
    
    # 응답 결과 (JSON)
    data = response.json()
    
    # 검색 결과의 'items' 키 아래에 결과 목록이 존재
    items = data.get('items', [])

    # items에서 필요한 데이터 추출 후 리스트에 저장
    for item in items:
        results.append({
            "title": item["title"],
            "link": item["link"],
            "description": item["description"],
            "bloggername": item["bloggername"],
            "bloggerlink": item["bloggerlink"],
            "postdate": item["postdate"]
        })

# 판다스 데이터프레임으로 변환
df = pd.DataFrame(results)

# 결과 확인
df

In [ ]:
# 데이터프레임을 CSV 파일로 저장하기
csv_filename = "naver_blog_search_result.csv"
df.to_csv(csv_filename, index=False)

# 저장한 CSV 파일을 불러와서 확인하기
loaded_df = pd.read_csv(csv_filename)
loaded_df.head()

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans

# title, description, bloggername 세 개 컬럼을 공백으로 연결
df['text'] = df['title'] + " " + df['description'] + " " + df['bloggername']
vectorizer = TfidfVectorizer(
    # max_features=1000,   # 필요한 경우 최대 피처 개수 제한
)
X = vectorizer.fit_transform(df['text'])

# ---------------------------
# 토픽 모델링(LDA)
# ---------------------------
# n_components: 추출할 토픽(주제) 개수
lda = LatentDirichletAllocation(n_components=4, random_state=42)
lda.fit(X)

# 문서별 토픽 분포
doc_topic_dists = lda.transform(X)  # shape: (문서 수, 토픽 개수)

# 각 토픽별 단어 분포
topic_word_dists = lda.components_  # shape: (토픽 개수, 단어(피처) 수)

# 토픽별 주요 단어 살펴보기
feature_names = vectorizer.get_feature_names_out()
def print_top_words(model, feature_names, n_top_words=10):
    for idx, topic in enumerate(model.components_):
        # 중요도가 높은(가중치가 큰) 단어 순으로 정렬
        top_features_idx = topic.argsort()[::-1][:n_top_words]
        top_features = [(feature_names[i], topic[i]) for i in top_features_idx]
        print(f"Topic #{idx}:")
        for word, score in top_features:
            print(f"  {word}: {score:.4f}")
        print()

print("==== LDA 토픽별 상위 단어 ====")
print_top_words(lda, feature_names, 5)


# ---------------------------------------
# 문서별 토픽 분포를 별도 데이터프레임으로 만들기
# ---------------------------------------
# 토픽 개수만큼 컬럼명 생성 (예: topic0, topic1, ...)
topic_cols = [f"topic{i}" for i in range(doc_topic_dists.shape[1])]
df_topic_dist = pd.DataFrame(doc_topic_dists, columns=topic_cols)

# title 칼럼을 맨 앞에 추가
df_topic_dist.insert(0, "title", df["title"])

# 결과 확인
print("==== 문서별 토픽 분포 데이터프레임 ====")
df_topic_dist

In [ ]:
df_topic_dist[['topic0', 'topic1', 'topic2', 'topic3']].sum(axis=1).describe()

In [ ]:
df_topic_dist.describe()

In [ ]:
# ---------------------------
# 클러스터링(K-Means)
# ---------------------------
# n_clusters: 클러스터 개수
kmeans = KMeans(n_clusters=4, random_state=42)
kmeans.fit(X)

# 각 문서(행)에 대한 군집(label)
labels = kmeans.labels_
df['cluster'] = labels

# 결과 확인
print("==== 클러스터링 결과 ====")
print(df[['title','cluster']])


# ---------------------------
# 클러스터별 상위 키워드 10개 출력
# ---------------------------
# (X는 (문서 수 x 단어 수) 형태의 TF-IDF 행렬)
# 각 클러스터의 주요 단어를 찾기 위해
# "클러스터 소속 문서들의 TF-IDF를 평균" 낸 뒤, 상위 키워드를 추출
n_top_keywords = 10
for cluster_id in range(kmeans.n_clusters):
    # 클러스터에 속한 문서 인덱스
    cluster_indices = np.where(labels == cluster_id)
    # 해당 클러스터 문서들의 TF-IDF 행렬만 추출
    cluster_docs_tfidf = X[cluster_indices]
    # 문서들의 평균 TF-IDF (shape: (1, 피처수))
    mean_tfidf = cluster_docs_tfidf.mean(axis=0)
    # sparse -> numpy array 변환
    mean_tfidf = mean_tfidf.A1
    
    # 평균 TF-IDF가 큰 순으로 정렬 후 상위 n_top_keywords 인덱스 추출
    top_indices = mean_tfidf.argsort()[::-1][:n_top_keywords]
    top_features = feature_names[top_indices]
    top_scores = mean_tfidf[top_indices]
    
    print(f"\n=== 클러스터 {cluster_id}의 상위 키워드 {n_top_keywords}개 ===")
    for rank, (feature, score) in enumerate(zip(top_features, top_scores), start=1):
        print(f"{rank}. {feature} (TF-IDF: {score:.4f})")